In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy as sp
import csv
from sklearn.model_selection import train_test_split,KFold,GridSearchCV
from sklearn.linear_model import SGDClassifier
from sklearn.decomposition import PCA,IncrementalPCA, FactorAnalysis
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.model_selection import validation_curve
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import precision_recall_curve

plt.rcParams['figure.figsize'] = [10, 5]
%matplotlib inline

from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestClassifier


In [ ]:
from sklearn import preprocessing

In [2]:
from tensorflow.python.keras.applications import ResNet50
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, Flatten, GlobalAveragePooling2D
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.python import keras
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, Flatten, Conv2D, Dropout

In [14]:
apptrain=pd.read_csv("modedapptrain.csv")
apptest=pd.read_csv("modedapptest.csv")
apptrain.drop("Unnamed: 0", axis=1, inplace=True)
apptest.drop("Unnamed: 0", axis=1, inplace=True)

In [15]:
apptrain_encoded=pd.read_csv("apptrain_encoded.csv")
apptest_encoded=pd.read_csv("apptest_encoded.csv")
apptrain_encoded.drop("Unnamed: 0",axis=1,inplace=True)
apptest_encoded.drop("Unnamed: 0",axis=1,inplace=True)

In [18]:

'''apptrain.describe()
apptest.head()
apptest.describe()'''

apptrain.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [21]:
X=apptrain_encoded.iloc[:,:248]
y=apptrain_encoded.iloc[:,248:249]

# random forest

In [ ]:
sm=SMOTE(random_state=15,ratio='minority')
x_res,y_res=sm.fit_sample(X=apptrain.drop("TARGET",axis=1),y=apptrain["TARGET"])

In [6]:
rfclf = RandomForestClassifier(n_estimators=256,random_state=12)

In [8]:
rfclf.fit(x_res, y_res)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=256, n_jobs=1,
            oob_score=False, random_state=12, verbose=0, warm_start=False)

In [9]:
probs = rfclf.predict_proba(apptest_encoded)

In [ ]:
Submissionohe=pd.DataFrame({'SK_ID_CURR':apptest_encoded.SK_ID_CURR,'TARGET':probs[:,1].round(1)})

# random forest with smote OHE

In [19]:
sm=SMOTE(random_state=15,ratio='minority')
x_res,y_res=sm.fit_sample(X=apptrain_encoded.drop("TARGET",axis=1),y=apptrain_encoded["TARGET"])

In [6]:
rfclf = RandomForestClassifier(n_estimators=256,random_state=12)

In [8]:
rfclf.fit(x_res, y_res)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=256, n_jobs=1,
            oob_score=False, random_state=12, verbose=0, warm_start=False)

In [9]:
probs = rfclf.predict_proba(apptest_encoded)

In [ ]:
preds = probs[:,1]
fpr, tpr, threshold = roc_curve(y_test1, preds)
roc_auc = auc(fpr, tpr)

In [ ]:
plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')

In [42]:
probs

array([[ 0.75      ,  0.25      ],
       [ 0.4765625 ,  0.5234375 ],
       [ 0.62890625,  0.37109375],
       ..., 
       [ 0.8828125 ,  0.1171875 ],
       [ 0.80078125,  0.19921875],
       [ 0.58984375,  0.41015625]])

In [10]:
Submissionohe=pd.DataFrame({'SK_ID_CURR':apptest_encoded.SK_ID_CURR,'TARGET':probs[:,1].round(1)})

In [12]:
Submissionohe.head()

,SK_ID_CURR,TARGET
0,100001,0.2
1,100005,0.5
2,100013,0.4
3,100028,0.1
4,100038,0.4


In [13]:
Submissionohe.to_csv("Submission2.csv",sep=',',encoding='utf-8')

# RANDOM FOREST simple ohe DATA

In [67]:
sm=SMOTE(random_state=15,ratio='minority')
x_res_ohe,y_res_ohe=sm.fit_sample(X=apptrain_encoded.drop("TARGET",axis=1),y=apptrain_encoded["TARGET"])

In [68]:
rfclf2 = RandomForestClassifier(n_estimators=1000,random_state=12)
rfclf2.fit(x_res_ohe, y_res_ohe)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=1,
            oob_score=False, random_state=12, verbose=0, warm_start=False)

In [ ]:
rfclf2.feature_importances_

In [ ]:
min_max_scaler = preprocessing.MinMaxScaler()
X_train_minmax = min_max_scaler.fit_transform(X_train)

In [69]:
probs2 = rfclf2.predict_proba(apptest_encoded)

In [70]:
Submission_OHE_1000Est=pd.DataFrame({'SK_ID_CURR':apptest_encoded.SK_ID_CURR,'TARGET':probs[:,1].round(1)})

In [71]:
Submission_OHE_1000Est.to_csv("Submission_OHE_1000Est.csv",sep=',',encoding='utf-8')

# KERAS ON OHE DATA

In [83]:
apptrain_encoded.head()

,SK_ID_CURR,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,...,WALLSMATERIAL_MODE_Mixed,WALLSMATERIAL_MODE_Monolithic,WALLSMATERIAL_MODE_Others,WALLSMATERIAL_MODE_Panel,"WALLSMATERIAL_MODE_Stone, brick",WALLSMATERIAL_MODE_Wooden,EMERGENCYSTATE_MODE_Missing,EMERGENCYSTATE_MODE_No,EMERGENCYSTATE_MODE_Yes,TARGET
0,100002,0,202500.0,406597.5,24700.5,351000.0,0.018801,-9461,-637,-3648.0,...,0,0,0,0,1,0,0,1,0,1
1,100003,0,270000.0,1293502.5,35698.5,1129500.0,0.003541,-16765,-1188,-1186.0,...,0,0,0,0,0,0,0,1,0,0
2,100004,0,67500.0,135000.0,6750.0,135000.0,0.010032,-19046,-225,-4260.0,...,0,0,0,0,0,0,1,0,0,0
3,100006,0,135000.0,312682.5,29686.5,297000.0,0.008019,-19005,-3039,-9833.0,...,0,0,0,0,0,0,1,0,0,0
4,100007,0,121500.0,513000.0,21865.5,513000.0,0.028663,-19932,-3038,-4311.0,...,0,0,0,0,0,0,1,0,0,0


In [9]:
apptrain_encoded.shape

(307511, 249)

In [26]:
apptrain_encoded.shape

(307511, 249)

In [20]:
kerasSeqModel1=Sequential()


In [80]:
kerasSeqModel1.add(Dense(128, activation='relu',input_shape=(248,)))
kerasSeqModel1.add(Dense(128, activation='relu'))
#kerasSeqModel1.add(Flatten())
kerasSeqModel1.add(Dropout(0.5))
kerasSeqModel1.add(Dense(128, activation='relu'))
kerasSeqModel1.add(Dense(1, activation='sigmoid'))

In [81]:
kerasSeqModel1.compile(loss=keras.losses.binary_crossentropy,
              optimizer='adam',
              metrics=['accuracy'])

In [83]:
kerasSeqModel1.fit(x, y,
          epochs=2,
          validation_split = 0.2)

Train on 246008 samples, validate on 61503 samples
Epoch 1/5
246008/246008 [==============================] - 94s 380us/step - loss: 0.2946 - acc: 0.9189 - val_loss: 0.2773 - val_acc: 0.9206
Epoch 2/5
246008/246008 [==============================] - 94s 382us/step - loss: 0.2814 - acc: 0.9189 - val_loss: 0.2774 - val_acc: 0.9206
Epoch 3/5
246008/246008 [==============================] - 94s 384us/step - loss: 0.2814 - acc: 0.9189 - val_loss: 0.2774 - val_acc: 0.9206
Epoch 4/5
246008/246008 [==============================] - 95s 386us/step - loss: 0.2814 - acc: 0.9189 - val_loss: 0.2773 - val_acc: 0.9206
Epoch 5/5
246008/246008 [==============================] - 93s 378us/step - loss: 0.2814 - acc: 0.9189 - val_loss: 0.2773 - val_acc: 0.9206


In [ ]:
kerasprobs=kerasSeqModel1.predict_proba(apptest_encoded)

# Light GBM

In [72]:
import lightgbm as lgb

In [ ]:
x_res_ohe,y_res_ohe=sm.fit_sample(X=apptrain_encoded.drop("TARGET",axis=1),y=apptrain_encoded["TARGET"])

In [76]:
train_data = lgb.Dataset(data=apptrain_encoded.drop("TARGET",axis=1),label=apptrain_encoded["TARGET"])

In [74]:
param = {'num_leaves':32, 'num_trees':1000, 'objective':'binary'}
param['metric'] = ['roc_curve','binary_logloss']

In [77]:
num_round = 10
bst = lgb.train(param, train_data, num_round)

In [78]:
ypred = bst.predict(apptest_encoded)

In [81]:
Submission_OHE_LGBM=pd.DataFrame({'SK_ID_CURR':apptest_encoded.SK_ID_CURR,'TARGET':ypred.round(1)})

In [82]:
Submission_OHE_LGBM.to_csv("Submission_OHE_LGBM.csv",sep=',',encoding='utf-8')